<a href="https://colab.research.google.com/github/Gurushankar2025/Langchain-rag-projects/blob/main/pdfchatbot/Pdf_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Packages and Library**

In [ ]:
!pip install langchain_community
!pip install transformers

In [ ]:
!pip install wikipedia

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain_huggingface

In [ ]:
!pip install pypdf

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install -U bitsandbytes

In [1]:
!pip install -U accelerate

### **Import Libraries**

In [ ]:
#Library for Knowledgebase,vectorstore,documentloader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import FAISS

#Library for LLM and embedding model wrappers from huggingface
from langchain_huggingface import HuggingFaceEmbeddings,HuggingFaceEndpoint,HuggingFacePipeline,ChatHuggingFace

#library for agent[React agent]
from langchain.tools import WikipediaQueryRun
from langchain.agents import create_react_agent
from langchain.agents import AgentType

In [ ]:
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_use_double_quant=True,
# )

### **Vector Store**

In [ ]:
loader = PyPDFLoader('/content/Class_10_Mathematics_English_Medium-2024_Edition-www.tntextbooks.in.pdf')

docs = loader.load()



In [ ]:
# loader1 = PyPDFLoader('/content/namma_kalvi_10th_maths_guide_em_219527.pdf')
# docs1 = loader1.load()

In [ ]:
loader2 = PyPDFLoader('/content/10th_Science_EM_Text_www.tntextbooks.in.pdf')
docs2 = loader2.load()

In [ ]:
spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docchunks = spliter.split_documents(docs)
docchunks2 = spliter.split_documents(docs2)
# docchunks1 = spliter.split_documents(docs1)

In [ ]:
embbeding = HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
allchunks = docchunks+docchunks2

In [ ]:
vs = FAISS.from_documents(documents=allchunks,embedding=embbeding)

### **LLM MODEL**

In [ ]:
import torch
device_flag = 1 if torch.cuda.is_available() else 0

In [ ]:
# from transformers import pipeline
# pipe = pipeline('text2text-generation',model='facebook/mbart-large-50-many-to-many-mmt',device=device_flag)
# llm=HuggingFacePipeline(pipeline=pipe)

#### **LLM Implemented using HuggingFaceEndpoint**

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_KEY']='your_HF_api_key'

In [ ]:
repo_id = 'deepseek-ai/DeepSeek-R1-0528'
LLM=HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.2,
    max_new_tokens=50,
    huggingfacehub_api_token='you _HF_api_key'

)

chat=ChatHuggingFace(llm=LLM)     #wrapping the LLm model with ChatHuggingFace in order to pass and get chat input (system and user messages) and output (ai message).

#### **LLM Implemented using HuggingFacePipeline**

In [ ]:


# llm = HuggingFacePipeline.from_model_id(
#     model_id="microsoft/phi-2",
#     task="text-generation",
#     pipeline_kwargs=dict(
#         max_new_tokens=512,

#     ),
# )

# chat_model = ChatHuggingFace(llm=llm)

In [ ]:
# from langchain_core.messages import (
#     HumanMessage,
#     SystemMessage,
# )

# messages = [
#     SystemMessage(content="You're a helpful assistant"),
#     HumanMessage(
#         content="What happens when an unstoppable force meets an immovable object?"
#     ),
# ]

# ai_msg = chat_model.invoke(messages)

In [ ]:
#system Message
template="""
You are StudyHelper, an AI assistant for students and workers. Always follow the Think → Act → Observe → Answer framework, but only show the final Answer and in details if math problems is involved in the final output. Give direct answers if simple, or use documents/tools if they provide better context.
"""

In [ ]:
# create chat template (system message(Instruction and behaviour prompt) and user message(prompt with user input variables))
from langchain_core.prompts import ChatPromptTemplate

chatprompt = ChatPromptTemplate(
    [("system",template),("user","Answer this Question: {query} with this document: {doc_contents}")]
)


### **AGENT**

In [ ]:
#Another way of creating a react agent
# from re import VERBOSE
# react_agent=create_react_agent(tools=tools,llm=llm,prompt=prompt)

In [ ]:
# load tools
from langchain.agents import load_tools
tools = load_tools(['wikipedia','llm-math'],llm=chat)

In [ ]:
#initialize agent
from langchain.agents import initialize_agent

react = initialize_agent(
    tools=tools,
    llm=chat,
    AgentType='chat-zero-shot-react-description',
    verbose=True
)

In [ ]:
# from langchain.agents import AgentExecutor

In [ ]:
# agent_exe = AgentExecutor.from_agent_and_tools(
#     agent = react_agent,
#     tools = tools,
#     verbose = True,
#     handle_parsing_error = True
# )

In [ ]:
# response=agent_exe.invoke({'query':query,'doc_contents':doc_contents})
# print(response['output'])

### **rag and agent execution pipeline**

In [ ]:

import re

def pdfchatbotexeflow(query):
    sim_docs = vs.similarity_search(query,k=3)      # for the query, does the similarity search and retrieve top 3 similar content for vector store
    doc_contents = " ".join([doc.page_content for doc in sim_docs])    # merge the retrieved documents page content (content present in that page presented in that document)
    formatprompt=chatprompt.format(query=query,doc_contents=doc_contents)  #pass the query and merged contents to the user prompt and format chat prompt into a single string(agent.run(only accepts string as input))
    response=react.run(formatprompt)   # pass the input string and get the final output here from llm to agent then here.
    cleaned = re.sub(r".*Final Answer\s*","",response,flags=re.DOTALL)  # used to fetch only the final answer and if thought or action of llm is presented in final output it will be  not be taken clean the ouptut as  mathematical expression extracted from pdf content will have \\ a
    cleaned = re.sub(r'\\\(|\\\)|\\\[|\\\]|\\\\', '', cleaned) # clean the ouptut as  mathematical expression extracted from pdf contents will have \\ so the llm also use these symbols in output expression.
    return cleaned.strip()  # return the output with no extra whitespaces

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(    #run a simple UI
    fn=pdfchatbotexeflow,   #agent and rag pipeline function
    title="PDF-CHAT-BOT",
    inputs=["text"],   #query
    outputs="text"     #ouput given by llm model
)

demo.launch()